In [ ]:
from utils import ltf_to_sent
import tqdm
import csv
import os
import spacy
import tqdm
nlp = spacy.load("en_core_web_trf")

# Main file

I used a custom fine-tuned Spacy model for NER. Here are relevant docs.

* Spacy-101: https://spacy.io/usage/spacy-101
* NER 101: https://spacy.io/usage/linguistic-features#named-entities
* NER architectures: https://spacy.io/api/architectures#parser
* Training: https://spacy.io/usage/training

In [1]:
from utils import ltf_to_sent
import tqdm
import csv
import os
import spacy
nlp = spacy.load("en_core_web_trf")

def get_raw_text_from_file(filepath, is_test=False):
  # /home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
    if is_test:
      basepath = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/test/data/ltf/"
    else: 
      basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/'
    filepath = os.path.join(basepath, filepath)
    filepath += ".ltf.xml"
    text_file = open(filepath, "r", encoding="utf-8")
    ltf_str = text_file.read()
    text = ltf_to_sent.ltf_to_plaintext(ltf_str)
    return text
      
def get_gold_labels(is_test=False):
  if is_test:
    gold_labels_path = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/gold_labels/gold.tab'
  else:
    gold_labels_path = 'gold_labels/edl.tab'
  lines = []
  with open(gold_labels_path, encoding="utf-8") as tsv:
    for line in csv.reader(tsv, dialect="excel-tab"): # dialect="excel-tab" or delimiter="\t" 
      lines.append(line)
  print("example lline:", lines[0])
  return lines


def prep_spacy_training():
  labels = get_gold_labels()
  
  # init variables
  first_doc_name = labels[0][3].split(':')[0]
  last_doc_filename = first_doc_name
  count = 0 
  num_span_is_none = 0
  doc = nlp(get_raw_text_from_file(first_doc_name))
  doc.ents = []
  db = spacy.tokens.DocBin()

  
  # iterate thru all labels, saving docs as we go
  for _, num, span_text, doc_name, ent_gold, annot_list, NAM_type, const in tqdm.tqdm(labels):
    curr_doc_filename = doc_name.split(':')[0]
    if curr_doc_filename != last_doc_filename:
      # save the last spacy.doc, and load in the next spacy.doc. 
      last_doc_filename = curr_doc_filename
      db.add(doc)
      doc = nlp(get_raw_text_from_file(curr_doc_filename))
      doc.ents = []
      print("starting new doc")
      print("Doc: ", doc)
      return
    
    # iterate thru the labels and set the spans.
    start_char, end_char = doc_name.split(':')[1].split('-')[0], doc_name.split(':')[1].split('-')[1]

    print("text:", span_text)
    print("ent_gold:", ent_gold)
    print("annot_list:", annot_list)
    print(doc_name)
    print(curr_doc_filename)
    print(start_char, end_char)

    print("Doc ents:", doc.ents)
    
    span = doc.char_span(int(start_char), int(end_char)+1, label=annot_list, alignment_mode="contract")
    print("span:",span)
    print("span type:", type(span))
    if span is None:
      num_span_is_none += 1
      continue
    if type(doc.ents) is not "NoneType":
      doc.ents = list(doc.ents) + [span]
    else:
      doc.ents = [span]
    
    if count > 3:
      break
    count += 1
  # end loop 
  db.to_disk("./train_concat_annot_list.spacy")
  print("⚠️ num_span_is_none:",num_span_is_none)

prep_spacy_training() 

<>:81: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:81: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/home/kastan/utils/miniconda3/envs/nlp_v2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2912301/2742994035.py:81: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if type(doc.ents) is not "NoneType":


#### Data created ^^ now run training

cd spacy_train

python -m spacy train ./config.cfg --output ./output_trained_pipeline --verbose --gpu-id 0

In [ ]:
""" Get most common annotation in train set. """

def most_frequent(List):
    counter = 0
    num = List[0]
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
    return num

only_labels = []
for item in get_gold_labels(is_test=True):
    first, sec, third, fourth, fifth, sixth, seventh, eighth = item
    only_labels.append(sixth)
    
print(most_frequent(only_labels))

In [2]:
""" Construct TEST DATA .spacy """
from utils import ltf_to_sent
import tqdm
import csv
import os
import spacy
nlp = spacy.load("en_core_web_trf")

def print_test_file():      
  db = spacy.tokens.DocBin()
  basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/'
  for file in tqdm.tqdm(os.listdir(basepath)):
    if file.endswith(".ltf.xml"):
      filepath = os.path.join(basepath, file)
      # print(filepath)
      text_file = open(filepath,"r",encoding="utf-8")
      ltf_str = text_file.read()
      raw_text = ltf_to_sent.ltf_to_plaintext(ltf_str)
      print(raw_text)
    break

def make_test_docs():      
  db = spacy.tokens.DocBin()
  basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/test/data/ltf/'
  for file in tqdm.tqdm(os.listdir(basepath)):
    if file.endswith(".ltf.xml"):
      filepath = os.path.join(basepath, file)
      # print(filepath)
      text_file = open(filepath,"r",encoding="utf-8")
      ltf_str = text_file.read()
      raw_text = ltf_to_sent.ltf_to_plaintext(ltf_str)
      # print(raw_text)
      doc = nlp(raw_text)
      doc.ents = []
      db.add(doc)

  db.to_disk("./test_docs.spacy")
  return db

print_test_file()
# make_test_docs()

""" Helpers for main prediction """

def filter_doc_to_only_mentions(doc):
  """ 
  doc: spacy doc
  """
  mentions = get_mentions_list()
  ents = []
  for ent in doc.ents:
    # make lowercase comparisons
    if str(ent.label_).lower() in mentions:
      ents.append(ent)
  doc.ents = ents
  return doc

def get_mentions_list():
  gold_labels_path = 'gold_labels/mentions.tab'
  lines = []
  with open(gold_labels_path, encoding="utf-8") as tsv:
    lines = list(csv.reader(tsv, dialect="excel-tab"))
  return lines

def write_mentions_list_to_file(final_pred_list):
  # mentions = get_mentions_list()
  with open('final_pred_list__PER_defult.tab', 'w', encoding="utf-8") as f:
    for item in final_pred_list:
      f.write("%s\n" % item)
  return

In [3]:
""" MAIN PREDICTIONS """

def make_predictions_main():
  """ Main driver function making score-compatible predictions. """
  final_pred_list = []
  mentions = get_mentions_list()

  # Load my custom model.  
  custom_model = spacy.load("./spacy_train/output_trained_pipeline/model-best")
  path = "./test_docs.spacy"
  path = "spacy_train/train_data_with_ents.spacy"
  
  # init variables
  first_doc_name = mentions[0][1].split(':')[0]
  last_doc_filename = first_doc_name
  count = 0
  curr_doc_annotation_counter = 0
  doc = custom_model(get_raw_text_from_file(first_doc_name, is_test=True))
  # print("first_test_doc.ents", list(doc.ents))
  
  for phrase, doc_name, NAM_type in tqdm.tqdm(mentions):
    curr_doc_filename = doc_name.split(':')[0]
    # print(phrase, doc_name, NAM_type)
    if curr_doc_filename != last_doc_filename:
      # save the last spacy.doc, and load in the next spacy.doc. 
      last_doc_filename = curr_doc_filename
      curr_doc_annotation_counter = 0
      doc = custom_model(get_raw_text_from_file(curr_doc_filename, is_test=True))
      print("starting new doc")
      
    # iterate thru the labels and set the spans.
    start_char, end_char = doc_name.split(':')[1].split('-')[0], doc_name.split(':')[1].split('-')[1]
    
    main_ent_type_pred = "PER" # default prediction, if no other type is found.
    
    """make main predictions."""
    # remove used ents so we don't double count (order matters)
    remaining_ents = [] 
    not_yet_found = True
    for ent in list(doc.ents):
      if phrase == ent.text and not_yet_found:
        # print("✅ Predicted: ", ent.label_, " ----- Actual:", phrase)
        main_ent_type_pred = ent.label_
        not_yet_found = False
      else:
        remaining_ents.append(ent)
    doc.ents = remaining_ents
      
    pred = f'NIST\tannotation-{curr_doc_annotation_counter}\t{phrase}\t{doc_name}\t{"other phrase I dont understand"}\t{main_ent_type_pred}\t{NAM_type}\t1.0'
    # print(pred)
    final_pred_list.append(pred)
  
  return final_pred_list


final_pred_list = make_predictions_main()
write_mentions_list_to_file(final_pred_list)

✅ Predicted:  PER;PER.SituationalRole.Suspect  ----- Actual: O.J. Simpson
✅ Predicted:  PER;PER.Professional;PER.Professional.HealthProfessional  ----- Actual: knife
✅ Predicted:  WEA;WEA.Gun;WEA.Gun.Rifle  ----- Actual: knife
✅ Predicted:  FAC;FAC.Building  ----- Actual: estate
✅ Predicted:  PER;PER.Farmer  ----- Actual: Ronald Goldman
✅ Predicted:  ORG;ORG.CommercialOrganization;ORG.CommercialOrganization.NewsAgency  ----- Actual: TMZ
✅ Predicted:  WEA;WEA.Gun;WEA.Gun.Rifle  ----- Actual: knife
✅ Predicted:  WEA;WEA.Gun;WEA.Gun.Rifle  ----- Actual: knife
✅ Predicted:  ORG;ORG.CommercialOrganization;ORG.CommercialOrganization.NewsAgency  ----- Actual: TMZ
✅ Predicted:  WEA;WEA.Gun;WEA.Gun.Rifle  ----- Actual: knife
✅ Predicted:  PER;PER.Laborer  ----- Actual: worker
✅ Predicted:  LOC;LOC.Region  ----- Actual: perimeter
✅ Predicted:  Document;Document.Report  ----- Actual: Simpson
✅ Predicted:  FAC;FAC.Building  ----- Actual: estate
✅ Predicted:  PER  ----- Actual: he
✅ Predicted:  FAC

: 

## Score results (see `./kvday2_FINAL_eval_scores` for my final results)

```bash
# My evaluation
python score_submission.py \
  ../../input/log_specifications.txt \
  /home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/gold_labels/edl.tab \
  /home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/final_pred_list.tab \
  ./kas_test_1_scores
```